<a href="https://colab.research.google.com/github/salarbalou/Data_Analysis_Projects/blob/main/HeatDiffusionProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.linalg import solve_banded
from scipy.optimize import newton_krylov
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='plotly_dark'

In [ ]:
r = 0.50
nx=100
nt=5000

In [ ]:
x=np.arange(nx)

In [ ]:
T=np.full(nx,300.)
T[0]=350
T[-1]=400
fig=make_subplots()
fig.add_scatter(x=x, y=T, name='initial')
for i in range(nt+1):
    T[1:-1]=T[1:-1]+r*(T[2:]- 2*T[1:-1]+T[:-2])
    if i % 100 == 0:
        fig.add_scatter(x=x,y=T,name=f'{i}')
fig.update_layout(width=800,height=600)

In [ ]:
nt=1000
nx=100
T=np.full(nx,300.)
r=10
T[0]=350
T[-1]=400
A=np.zeros((3,nx-2))
A[0,:]=-r
A[1,:]=(1+2*r)
A[2,:]=-r
b = r*T[2:] + (1-2*r)*T[1:-1] + r*T[:-2]
b[0] = b[0] + r*T[0]
b[-1] = b[-1] + r*T[-1]
fig2=make_subplots()
fig2.add_scatter(x=x, y=T, name='initial')
for i in range(nt+1):
    T[1:-1]=solve_banded((1,1),A,b)
    b = r*T[2:] + (1-2*r)*T[1:-1] + r*T[:-2]
    b[0] = b[0] + r*T[0]
    b[-1] = b[-1] + r*T[-1]
    if i % 100 == 0:
        fig2.add_scatter(x=x,y=T,name=f'{i}')
fig2.update_layout(width=800,height=600)

In [ ]:
nx,ny = 20,20
dx, dy = 1./(nx-1), 1./(ny-1)
x,y=np.mgrid[0:1:nx*1j,0:1:ny*1j]

In [ ]:
Ttop,Tbottom = 350., 250.
Tleft, Tright = 300., 400.
T=np.full((nx,ny), (Ttop+Tbottom+Tleft+Tright)/4)
T[0,:]=Ttop
T[-1,:]=Tbottom
T[:,0]=Tleft
T[:,-1]=Tright
qk = 1000.

def hde2d(Tunk):
    T[1:-1,1:-1]=Tunk
    d2x = (T[2:,1:-1] - 2*T[1:-1,1:-1] + T[:-2,1:-1])/dx**2
    d2y = (T[1:-1,2:] - 2*T[1:-1,1:-1] + T[1:-1,:-2])/dy**2
    return d2x+d2y + qk

T[1:-1,1:-1]=newton_krylov(hde2d, T[1:-1,1:-1])

In [ ]:
fig2 = make_subplots()
fig2.add_trace(go.Surface(x=x,y=y,z=T))
for i in range(nx):
    fig2.add_trace(go.Scatter3d(x=x[i,:],y=y[i,:],z=T[i,:], mode='lines', line_color='gray'))
for i in range(ny):
    fig2.add_trace(go.Scatter3d(x=x[:,i],y=y[:,i],z=T[:,i], mode='lines', line_color='gray'))


fig2.update_layout(width=800,height=600, showlegend=False)

In [ ]:
nr, ntheta = 10, 10
rmin, rmax = 1., 4.
thetalo, thetahi = 0, np.pi/2
dr, dtheta = (rmax-rmin)/(nr-1), (thetahi-thetalo)/(ntheta-1)
r, theta = np.mgrid[rmin:rmax:nr*1j, thetalo:thetahi:ntheta*1j]
x,y = r*np.cos(theta), r*np.sin(theta)


In [ ]:
Trmin, Trmax = 250., 300.
Tthetalo, Tthetahi = 260., 280.

In [ ]:
T=np.full((nr,ntheta), (Trmin + Trmax + Tthetalo +Tthetahi)/4)
T[0,:], T[-1, :]=Trmin,Trmax
T[:,0], T[:,-1] = Tthetalo, Tthetahi

In [ ]:
def hdepolar(Tunk):
    T[1:-1,1:-1]=Tunk
    d2r = ((r[1:-1,1:-1]+dr/2)*(T[2:,1:-1] - T[1:-1,1:-1]) - (r[1:-1,1:-1]-dr/2)*(T[1:-1,1:-1] - T[:-2,1:-1]))/dr**2
    d2theta = (T[1:-1,2:] - 2*T[1:-1,1:-1] + T[1:-1,:-2])/dtheta**2 / r[1:-1,1:-1]

    return d2r+d2theta

In [ ]:
T[1:-1,1:-1]=newton_krylov(hdepolar, T[1:-1,1:-1])

In [ ]:
fig3 = make_subplots()
fig3.add_trace(go.Surface(x=x,y=y,z=T))
for i in range(nr):
    fig3.add_trace(go.Scatter3d(x=x[i,:],y=y[i,:],z=T[i,:], mode='lines', line_color='gray'))
for i in range(ntheta):
    fig3.add_trace(go.Scatter3d(x=x[:,i],y=y[:,i],z=T[:,i], mode='lines', line_color='gray'))


fig3.update_layout(width=800,height=600, showlegend=False)